## Notebook - Concentration en polluants atmosphériques - Département du Rhône (69)


### Connection au GIS et import des bibliothèques

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

from arcgis.mapping import WebMap
from arcgis.apps.dashboard import Dashboard
from arcgis.apps.dashboard import Header, RichText, Indicator, MapLegend, SidePanel
from arcgis.apps.dashboard import add_row, add_column

import pandas as pd
import os
import datetime
import shutil

#import logging
#logging.captureWarnings(True)

### Préparation des espaces de travail

#### Variables

In [ ]:
today = datetime.date.today().strftime("%y_%m_%d")
todayDisplay = datetime.date.today().strftime("%d/%m/%y")

polluants = ["no2","o3","pm25"]
concentrations=["conc_no2","conc_o3","conc_pm25"]

limites69 = '/arcgis/home/Limites_admin_69/admin-departement.shp'

#### Fichier principal

In [ ]:
def create_folder(folder_name: str) -> None:
    home_path = os.path.abspath('/arcgis/home')
    folder_path = os.path.join(home_path, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
    else:
        print(f"A folder named {folder_name} already exists in {home_path}.")

create_folder('Analyses_ATMO_{0}'.format(today))

#### Sous-fichiers

In [ ]:
def create_subfolder(folder_name: str) -> None:
    home_path = os.path.abspath('/arcgis/home/Analyses_ATMO_{0}'.format(today))
    folder_path = os.path.join(home_path, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
    else:
        print(f"A folder named {folder_name} already exists in {home_path}.")
        
subfolders = ['Points_Echantillonage_{0}'.format(today)] 

for concentration in concentrations : 
    subfolders.append('EBK_{1}_{0}'.format(today,concentration))
    subfolders.append('EBK_Clipped_{1}_{0}'.format(today,concentration))
    
for subfolder in subfolders :
    create_subfolder(subfolder)

#### GDB

In [ ]:
def create_fgdb(folder_path: str, fgdb_name: str) -> None:
    if not os.path.isdir(folder_path):
        os.mkdir(folder_path)
    arcpy.env.workspace = folder_path
    fgdb_path = os.path.join(folder_path, fgdb_name)
    if not arcpy.Exists(fgdb_name):
        arcpy.management.CreateFileGDB(folder_path, fgdb_name)
        print(f"Created file geodatabase: {fgdb_path}")
    else:
        print(f"A file geodatabase named {fgdb_name} already exists in {folder_path}.")

create_fgdb('/arcgis/home/Analyses_ATMO_{0}'.format(today), 'database.gdb')
arcpy.env.workspace = '/arcgis/home/Analyses_ATMO_{0}/database.gdb'.format(today)

### Préparation des données

#### Import du csv et interprétation en Pandas Dataframe

In [ ]:
inputDataset = '/arcgis/home/Analyses_ATMO_23_04_20/Indice_ATMO_2023_04_17_utf8.csv'

In [ ]:
df = pd.read_csv(inputDataset,sep=',', low_memory=False, encoding="utf8")
df

#### Nettoyage des données (département du Rhône + supression des valeurs manquantes)

In [ ]:
df["code_zone"] = df["code_zone"].astype(str)
df = df[df["code_zone"].str.startswith('69',na=False)]
df

In [ ]:
df = df.dropna()
df

#### Réécriture au format .csv

In [ ]:
cleanDataset = '/arcgis/home/Analyses_ATMO_{0}/Indice_ATMO_69_{0}_utf-8.csv'.format(today)
df.to_csv(cleanDataset, sep=';',encoding='utf-8')

### Traitements

#### Table vers points

In [ ]:
points = "/arcgis/home/Analyses_ATMO_{0}/Points_Echantillonage_{0}/Points_atmo_69_{0}".format(today)

In [ ]:
arcpy.management.XYTableToPoint(cleanDataset, points, "x_wgs84", "y_wgs84")

#### Interpolation des concentrations de polluants (NO2, O3 et PM2,5) et découpe aux limites du département

In [ ]:
for concentration in concentrations :
    
    EBKOutput = "EDK.lyr"
    EBKFilledOutput = r"/arcgis/home/Analyses_ATMO_{0}/EBK_{1}_{0}/EBK_{1}_{0}".format(today, concentration)
    ClippedOutput = r"/arcgis/home/Analyses_ATMO_{0}/EBK_Clipped_{1}_{0}/EBK_Clipped_{1}_{0}".format(today, concentration)
    
    arcpy.EmpiricalBayesianKriging_ga(points, concentration,EBKOutput)
    arcpy.GALayerToContour_ga(EBKOutput, "Filled_contour", EBKFilledOutput,
                          "Presentation", "GEOMETRIC_INTERVAL", 30, "")
    
    arcpy.analysis.Clip(EBKFilledOutput, limites69, ClippedOutput)

### Partage des données sur le portail

In [ ]:
foldersToPublish = ['Points_Echantillonage_{0}'.format(today),'EBK_Clipped_conc_no2_{0}'.format(today),'EBK_Clipped_conc_o3_{0}'.format(today),'EBK_Clipped_conc_pm25_{0}'.format(today)] 

couches = []

for folder in foldersToPublish :
    archive = shutil.make_archive('/arcgis/home/Analyses_ATMO_{0}/{1}'.format(today,folder),'zip','/arcgis/home/Analyses_ATMO_{0}/{1}'.format(today,folder))
    shpfile = gis.content.add({}, archive)
    published_service = shpfile.publish()
    shpfile.move(gis.users.me.folders[0])
    published_service.move(gis.users.me.folders[0])
    couches.append(published_service.id)
    display(published_service)
    
del couches[0]

### Application de la symbologie, création des cartes et du Dashboard

#### Renderers

In [ ]:
rendererno2 = {"renderer": "autocast", #This tells python to use JS autocasting
               "type": "classBreaks",
               "field":"Value_Max",
               "minValue":0}

rendererno2["visualVariables"]= [{   "type": "colorInfo",
                                        "field": "Value_Max",
                                        "stops": [  {
                                                        "value": 10,
                                                        "color": [19,255,202,130],
                                                        "label": "<10"
                                                    },
                                                    {
                                                        "value": 20,
                                                        "color": [176,255,133,130],
                                                        "label": "20"
                                                    },
                                                    {
                                                        "value": 30,
                                                        "color": [242,254,30,130],
                                                        "label": "30"
                                                    },
                                                    {
                                                        "value": 40,
                                                        "color": [255,167,19,130],
                                                        "label": "40"
                                                    },
                                                    {
                                                        "value": 50,
                                                        "color": [254,35,10,130],
                                                        "label": ">50"
                                                    }]
                                    }]
rendererno2["classBreakInfos"] = [{   "symbol": {
                                            "color": [170,170,170,0],
                                            "outline": {
                                                "color": [153,153,153,0],
                                                "width": 0.375,
                                                "type": "esriSLS",
                                                "style": "esriSLSSolid"
                                            },
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"
                                        },
                                        "classMaxValue": 9007199254740991
                                    }]                

In [ ]:
renderero3 = {"renderer": "autocast", #This tells python to use JS autocasting
               "type": "classBreaks",
               "field":"Value_Max",
               "minValue":0}

renderero3["visualVariables"]= [{   "type": "colorInfo",
                                        "field": "Value_Max",
                                        "stops": [  {
                                                        "value": 70,
                                                        "color": [19,255,202,130],
                                                        "label": "<70"
                                                    },
                                                    {
                                                        "value": 80,
                                                        "color": [176,255,133,130],
                                                        "label": "80"
                                                    },
                                                    {
                                                        "value": 90,
                                                        "color": [242,254,30,130],
                                                        "label": "90"
                                                    },
                                                    {
                                                        "value": 100,
                                                        "color": [255,167,19,130],
                                                        "label": "100"
                                                    },
                                                    {
                                                        "value": 120,
                                                        "color": [254,35,10,130],
                                                        "label": ">120"
                                                    }]
                                    }]
renderero3["classBreakInfos"] = [{   "symbol": {
                                            "color": [170,170,170,0],
                                            "outline": {
                                                "color": [153,153,153,0],
                                                "width": 0.375,
                                                "type": "esriSLS",
                                                "style": "esriSLSSolid"
                                            },
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"
                                        },
                                        "classMaxValue": 9007199254740991
                                    }]                

In [ ]:
rendererpm25 = {"renderer": "autocast", #This tells python to use JS autocasting
               "type": "classBreaks",
               "field":"Value_Max",
               "minValue":0}

rendererpm25["visualVariables"]= [{   "type": "colorInfo",
                                        "field": "Value_Max",
                                        "stops": [  {
                                                        "value": 0,
                                                        "color": [19,255,202,130],
                                                        "label": "0"
                                                    },
                                                    {
                                                        "value": 5,
                                                        "color": [176,255,133,130],
                                                        "label": "5"
                                                    },
                                                    {
                                                        "value": 10,
                                                        "color": [242,254,30,130],
                                                        "label": "10"
                                                    },
                                                    {
                                                        "value": 15,
                                                        "color": [255,167,19,130],
                                                        "label": "15"
                                                    },
                                                    {
                                                        "value": 25,
                                                        "color": [254,35,10,130],
                                                        "label": ">25"
                                                    }]
                                    }]
rendererpm25["classBreakInfos"] = [{   "symbol": {
                                            "color": [170,170,170,0],
                                            "outline": {
                                                "color": [153,153,153,0],
                                                "width": 0.375,
                                                "type": "esriSLS",
                                                "style": "esriSLSSolid"
                                            },
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"
                                        },
                                        "classMaxValue": 9007199254740991
                                    }]                

#### Création des cartes

In [ ]:
carte = gis.map('Lyon')
carte.basemap='topo-vector'

renderers = [rendererno2,renderero3,rendererpm25]

i=0
webmapsIDs = []

for couche in couches :
    polluant = gis.content.get("{0}".format(couche))
    carte.add_layer(polluant.layers[0],
               { "type": "FeatureLayer",
                 "renderer" : renderers[i]})
    
    carte.zoom_to_layer(polluant.layers[0])
    
    webmap_properties = {'title':'Analyses de_concentration de {1} du {0}'.format(todayDisplay,polluants[i]),
                     'snippet': 'Concentration atmospherique de {1} selon les releves ATMO du {0}'.format(todayDisplay,polluants[i]),
                     'tags':['automation', 'python','demo']}

    webmap_item = carte.save(webmap_properties)
    webmap_item.move(gis.users.me.folders[0])
    display(webmap_item)
    webmapsIDs.append(webmap_item.id)
    
    carte.remove_layers()
    
    i+=1

#### Création du Dashboard

In [ ]:
carte_search = gis.content.search("{0}".format(webmapsIDs[1]))[0]
carte_search

In [ ]:
#carte no2
carte_search = gis.content.search("{0}".format(webmapsIDs[0]))[0]
carteid = carte_search.id
carte_get = gis.content.get("{0}".format(carteid))
no2Webmap = WebMap(carte_get)

no2Webmap.width=1
no2Webmap.height=0.73
no2Webmap.basemap_switcher = True
no2Webmap.search = True
    #légende
legendeno2 = MapLegend(no2Webmap)
legendeno2.height=0.2
    #texte
textno2 = RichText("<h2>Concentration de NO2 (µg.m-3)</h2>")
textno2.height=0.07


#carte o3
carte_search = gis.content.search("{0}".format(webmapsIDs[1]))[0]
carteid = carte_search.id
carte_get = gis.content.get("{0}".format(carteid))
o3Webmap = WebMap(carte_get)
o3Webmap.width=1
o3Webmap.height=0.73
o3Webmap.basemap_switcher = True
o3Webmap.search = True
    #légende
legendeo3 = MapLegend(o3Webmap)
legendeo3.height=0.2
    #texte
texteo3 = RichText("<h2>Concentration d'O3 (µg.m-3)</h2>")
texteo3.height=0.07


#carte pm25
carte_search = gis.content.search("{0}".format(webmapsIDs[2]))[0]
carteid = carte_search.id
carte_get = gis.content.get("{0}".format(carteid))
pm25Webmap = WebMap(carte_get)

pm25Webmap.width=1
pm25Webmap.height=0.73
pm25Webmap.basemap_switcher = True
pm25Webmap.search = True
    #légende
legendepm25 = MapLegend(pm25Webmap)
legendepm25.height=0.2
    #texte
textepm25 = RichText("<h2>Concentration de PM2.5 (µg.m-3)</h2>")
textepm25.height=0.07

#entête
entete = Header(title='Analyses de concentrations atmosphériques de NO2, O3 et PM2.5',
                subtitle='Selon interpolation des relevés ATMO du {0}'.format(todayDisplay),
                margin=True,
                size='large')

#création et sauvegarde du Dashboard
dashboard = Dashboard()
dashboard.header = entete
dashboard.theme = "light"
dashboard.layout = add_column([add_row([textno2,no2Webmap,legendeno2]),add_row([texteo3,o3Webmap,legendeo3]),add_row([textepm25,pm25Webmap,legendepm25])])
dashboard.save('Dashboard Analyses Atmosphériques du {0}'.format(todayDisplay),
                description="Dashboards des analyses de concentration atmosphériques dans le Rhône de NO2, O3 et PM2.5 selon une interpolations des relevés ATMO du {0}".format(todayDisplay),
                tags='ATMO, Demo',
                overwrite=True)

### Partage dans le groupe et notification des membres

#### Partage

In [ ]:
groupe = gis.groups.search('title: ATMO 69', max_groups=1)
dashboard = gis.content.search('Dashboard Analyses Atmosphériques du {0}'.format(todayDisplay))[0]
dashboard.move(gis.users.me.folders[0])

In [ ]:
dashboard.share(groups=groupe)

#### Notification

In [ ]:
#Lister les membres du groupe
membres = groupe[0].get_members()
#formatage des données
liste_membres = []
liste_notifs = []
for clef, membre in membres.items() :
    if clef !='owner':
        liste_membres.append(membre)
        
for admin in liste_membres[0] :
    liste_notifs.append(admin)
for user in liste_membres[1] :
    liste_notifs.append(user)
display(liste_notifs)

In [ ]:
gis.users.send_notification(liste_notifs, "Nouveau Dashboard disponible",#titre du message
                            "Le Dashboard d'analyses des concentrations atmosphériques en polluants NO2, O3 et PM2,5 dans le Rhône du {0} est disponible dans le groupe ATMO 69 (https://esrifrance.maps.arcgis.com/home/group.html?id=b3824c2fe84845f99db2d6e5f884a71f#overview)".format(todayDisplay), #contenu du message
                            "email" #à retirer pour avoir une notification sur le portail
                           )